In [12]:
import torch
from torch import nn
tensor = torch.rand(4)
print(tensor)

t = torch.flatten(tensor)
loss = nn.BCELoss()
# res = loss(t, torch.tensor([1]).float())

res = loss(t, torch.full(tuple(t.shape), fill_value=1).float())
print(torch.mean(-torch.log(tensor))) 
print(type(res))

tensor([0.4098, 0.3107, 0.1577, 0.4388])
tensor(1.1828)
<class 'torch.Tensor'>


In [13]:
import os
import sys
from pathlib import Path

script_dir = os.getcwd()
DATA_FOLDER = os.path.join(script_dir, 'training', 'data')
current = script_dir
while 'src' not in os.listdir(current):
    current = Path(current).parent

sys.path.append(str(current))
sys.path.append(os.path.join(str(current), 'FaceSpoofing'))


In [16]:
import shutil
import src.pytorch_modular.directories_and_files as dirf


RAW_DATA = os.path.join(DATA_FOLDER, 'raw')
if os.path.isdir(RAW_DATA):
    shutil.rmtree(RAW_DATA)

dirf.unzip_data_file(os.path.join(DATA_FOLDER, 'raw.zip'))
dirf.squeeze_directory(RAW_DATA)
# remove any non directories in RAW_DATA folder
# rename
os.rename(os.path.join(RAW_DATA, 'ClientRaw'), os.path.join(RAW_DATA, 'real'))
os.rename(os.path.join(RAW_DATA, 'ImposterRaw'), os.path.join(RAW_DATA, 'fake'))

initial_raw_data = os.listdir(RAW_DATA)
for f in initial_raw_data:
    pf = os.path.join(RAW_DATA, f)
    if not os.path.isdir(pf):
        os.remove(pf)

raw1 = os.path.join(DATA_FOLDER, 'raw', 'real')
raw0 = os.path.join(DATA_FOLDER, 'raw', 'fake')


initial_dirs1 = os.listdir(raw1)
initial_dirs0 = os.listdir(raw0)


In [17]:
for raw1_dir in initial_dirs1:
    path_dir = os.path.join(raw1, raw1_dir)
    if os.path.isdir(path_dir) and not str(raw1_dir).endswith('.db'):
        dirf.copy_directories(path_dir, raw1, copy=False, filter_directories=lambda x: not x.endswith('.db'))   
    else:
        os.remove(path_dir)
    # remove the directory at the end
    shutil.rmtree(path_dir)
        
for raw0_dir in initial_dirs0:
    path_dir = os.path.join(raw0, raw0_dir)

    if os.path.isdir(path_dir) and not str(raw0_dir).endswith('.db'):
        dirf.copy_directories(path_dir, raw0,copy=False, filter_directories=lambda x: not x.endswith('.db'))
    else:
        os.remove(path_dir)
    shutil.rmtree(path_dir)


In [18]:
# let's put everything together
TRAIN_DIR = os.path.join(DATA_FOLDER, 'train')
TEST_DIR = os.path.join(DATA_FOLDER, 'test')

if os.path.isdir(TRAIN_DIR):
    shutil.rmtree(TRAIN_DIR)
    os.makedirs(TRAIN_DIR)

if os.path.isdir(TEST_DIR):
    shutil.rmtree(TEST_DIR)
    os.makedirs(TEST_DIR)

In [19]:
# pass a portion of the dataset to the training dir
dirf.dataset_portion(RAW_DATA, TRAIN_DIR, portion=0.9, copy=False)
dirf.copy_directories(RAW_DATA, TEST_DIR, copy=False)

In [20]:
from FaceSpoofing.PixBiSupervision.training import preprocessing as pr, losses as ls
from  FaceSpoofing.PixBiSupervision.model import PiBiNet as binet,  DenseNetFE as dense
import importlib

importlib.reload(pr)
importlib.reload(ls)
importlib.reload(binet)
importlib.reload(dense)

pr.prepare_data(TRAIN_DIR)
pr.prepare_data(TEST_DIR)
# train_dl, test_dl = pr.prepare_dataloaders(TRAIN_DIR, TEST_DIR)

cuda
batch 2 processed !!
batch 3 processed !!
batch 4 processed !!
batch 5 processed !!
batch 6 processed !!
batch 7 processed !!
batch 8 processed !!
batch 9 processed !!
batch 10 processed !!
batch 11 processed !!
batch 12 processed !!
batch 13 processed !!
batch 14 processed !!
batch 15 processed !!
batch 16 processed !!
batch 17 processed !!
batch 18 processed !!
batch 19 processed !!
batch 20 processed !!
batch 21 processed !!
batch 22 processed !!
batch 23 processed !!
batch 24 processed !!
batch 25 processed !!
batch 26 processed !!
batch 27 processed !!
batch 28 processed !!
batch 2 processed !!
batch 3 processed !!
batch 4 processed !!
batch 5 processed !!
batch 6 processed !!
batch 7 processed !!
batch 8 processed !!
batch 9 processed !!
batch 10 processed !!
batch 11 processed !!
batch 12 processed !!
batch 13 processed !!
batch 14 processed !!
batch 15 processed !!
batch 16 processed !!
batch 17 processed !!
batch 18 processed !!
batch 19 processed !!


In [23]:
# prepare the data loaders
from src.pytorch_modular.data_loaders import create_dataloaders
train_dl, test_dl, classes_info = create_dataloaders(train_dir=TRAIN_DIR, 
                                       test_dir=TEST_DIR,
                                       train_transform=binet.DenseNetFeatureExtractor.default_transform,
                                       batch_size=256)

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
          [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           ...,
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
          [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
           [-

In [ ]:
# let's define the model
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
model = binet.PiBiNet()
loss = ls.pixelBinaryLoss()
optimizer = AdamW(params=model.parameters(), lr=10**-4, weight_decay=10**-5)
lr_scheduler = LinearLR(optimizer=optimizer, start_factor=1, end_factor=0.5, total_iters=100)

